# 分类中的泛化

:label:`chap_classification_generalization`



到目前为止，我们已经专注于如何通过训练具有多个输出和softmax函数的（线性）神经网络来解决多类分类问题。将我们模型的输出解释为概率预测，我们推导了交叉熵损失函数，该函数计算我们的模型（对于一组固定的参数）分配给实际标签的负对数似然。最后，我们将这些工具付诸实践，通过拟合训练集来调整我们的模型。然而，一如既往，我们的目标是学习*通用模式*，通过先前未见过的数据（测试集）进行经验评估。在训练集上的高准确性毫无意义。每当每个输入都是独一无二的时候（实际上对于大多数高维数据集来说确实如此），我们可以在第一个训练周期中仅仅记住数据集，并且在看到新图像时查找标签，从而在训练集上达到完美的准确性。然而，记忆与确切训练样本关联的确切标签并不能告诉我们如何分类新的样本。如果没有进一步的指导，我们可能不得不在遇到新样本时退回到随机猜测。

一些迫切的问题需要立即关注：

1. 我们需要多少测试样本来对我们的分类器在基础人群中的准确性给出良好的估计？
1. 如果我们不断重复地用同一测试集评估模型会发生什么？
1. 为什么我们应该期望将我们的线性模型拟合到训练集上会比我们简单的记忆方案表现得更好？


虽然 :numref:`sec_generalization_basics` 介绍了在回归上下文中的过拟合和泛化的基础知识，但本章将深入一点，介绍统计学习理论的一些基本思想。事实证明，我们通常可以*先验*保证泛化：对于许多模型，以及对于任何所需的泛化差距 $\epsilon$ 的上限，我们通常可以确定所需的一些样本数量 $n$，使得如果我们的训练集至少包含 $n$ 个样本，我们的经验误差将位于真实误差的 $\epsilon$ 范围内，*对于任何数据生成分布*。不幸的是，也发现虽然这种保证提供了一套深刻的知识构建块，但对于深度学习从业者而言，它们的实际效用有限。简而言之，这些保证表明，确保深度神经网络*先验*泛化需要大量荒谬的例子（也许万亿或更多），即使我们在关心的任务上发现，深度神经网络通常只需少得多的例子（数千）就能很好地泛化。因此，深度学习从业者经常完全放弃*先验*保证，而是采用过去在类似问题上表现良好的方法，并通过经验评估*事后*认证泛化。当我们到达 :numref:`chap_perceptrons` 时，我们将重新讨论泛化，并对试图解释为什么深度神经网络在实践中能够泛化的庞大科学文献进行简要介绍。

## 测试集

既然我们已经开始依赖测试集作为评估泛化误差的黄金标准方法，那么让我们从讨论这种误差估计的属性开始。让我们关注一个固定的分类器 $f$，而不必担心它是如何获得的。此外假设我们拥有一组全新的示例数据集 $\mathcal{D} = {(\mathbf{x}^{(i)},y^{(i)})}_{i=1}^n$，这些数据没有用于训练分类器 $f$。我们的分类器 $f$ 在 $\mathcal{D}$ 上的经验误差只是预测 $f(\mathbf{x}^{(i)})$ 与真实标签 $y^{(i)}$ 不一致的实例的比例，由以下表达式给出：

$$\epsilon_\mathcal{D}(f) = \frac{1}{n}\sum_{i=1}^n \mathbf{1}(f(\mathbf{x}^{(i)}) \neq y^{(i)}).$$

相比之下，*总体误差*是我们分类器在基础人群中（某个分布 $P(X,Y)$ 由概率密度函数 $p(\mathbf{x},y)$ 表征）不同意真实标签的*预期*比例：

$$\epsilon(f) =  E_{(\mathbf{x}, y) \sim P} \mathbf{1}(f(\mathbf{x}) \neq y) =
\int\int \mathbf{1}(f(\mathbf{x}) \neq y) p(\mathbf{x}, y) \;d\mathbf{x} dy.$$

虽然 $\epsilon(f)$ 是我们真正关心的数量，但我们不能直接观察它，就像我们不能直接观察大型人群中每个人的平均身高一样，而不需要测量每个人。我们只能基于样本来估计这个量。因为我们的测试集 $\mathcal{D}$ 在统计上代表了基础人群，我们可以将 $\epsilon_\mathcal{D}(f)$ 视为总体误差 $\epsilon(f)$ 的统计估计量。此外，因为我们感兴趣的量 $\epsilon(f)$ 是一个期望值（随机变量 $\mathbf{1}(f(X) \neq Y)$ 的期望），相应的估计量 $\epsilon_\mathcal{D}(f)$ 是样本均值，估计总体误差就是经典的均值估计问题，你可能在 :numref:`sec_prob` 中记得这一点。

概率论中的一个重要经典结果称为*中心极限定理*，保证了无论何时我们拥有 $n$ 个从任意分布中抽取的随机样本 $a_1, ..., a_n$，其均值为 $\mu$，标准差为 $\sigma$，则随着样本数量 $n$ 接近无穷大，样本均值 $\hat{\mu}$ 近似趋向于以真实均值为中心且标准差为 $\sigma/\sqrt{n}$ 的正态分布。这已经告诉我们一些重要的信息：当样本数量变大时，我们的测试误差 $\epsilon_\mathcal{D}(f)$ 应该以 $\mathcal{O}(1/\sqrt{n})$ 的速率接近真实误差 $\epsilon(f)$。因此，为了使我们的测试误差估计精确两倍，我们必须收集四倍大的测试集。为了将我们的测试误差减少一百倍，我们必须收集一万倍大的测试集。通常情况下，这样的 $\mathcal{O}(1/\sqrt{n})$ 速率往往是我们在统计学中所能希望的最佳情况。

现在我们知道了一些关于测试误差 $\epsilon_\mathcal{D}(f)$ 收敛到真实误差 $\epsilon(f)$ 的渐近率的信息，我们可以更详细地研究一些重要细节。回想一下感兴趣的随机变量 $\mathbf{1}(f(X) \neq Y)$ 只能取值 0 和 1，因此是一个伯努利随机变量，其特征参数表示它取值 1 的概率。在这里，1 意味着我们的分类器犯了错误，因此我们随机变量的参数实际上是真实误差率 $\epsilon(f)$。伯努利的方差 $\sigma^2$ 取决于其参数（这里为 $\epsilon(f)$），根据表达式 $\epsilon(f)(1-\epsilon(f))$。虽然 $\epsilon(f)$ 最初未知，但我们知道它不能大于 1。对该函数的一点调查表明，当真实误差率接近 0.5 时，我们的方差最高，而在接近 0 或接近 1 时可能会低得多。这告诉我们，我们对误差 $\epsilon(f)$ 的估计 $\epsilon_\mathcal{D}(f)$ 的渐近标准偏差（在选择 $n$ 个测试样本时）不能超过 $\sqrt{0.25/n}$。

如果我们忽略这一速率表征了测试集大小接近无穷大时的行为而不是当我们拥有有限样本时的行为，这告诉我们，如果我们希望我们的测试误差 $\epsilon_\mathcal{D}(f)$ 逼近总体误差 $\epsilon(f)$，使得一个标准偏差对应于 $\pm 0.01$ 的区间，那么我们应该收集大约 2500 个样本。如果我们想在这个范围内适应两个标准偏差并因此有 95% 的信心 $\epsilon_\mathcal{D}(f) \in \epsilon(f) \pm 0.01$，那么我们需要 10,000 个样本！

这正是机器学习中许多流行基准测试集的大小。你可能会惊讶地发现，每年有成千上万的应用深度学习论文发表，都在强调误差率改进 0.01 或更小的重要性。当然，当误差率远接近 0 时，0.01 的改进确实是个大事。


迄今为止，我们的分析的一个恼人特征是它实际上只告诉我们渐近性，即随着样本量趋于无穷大，$\epsilon_\mathcal{D}$ 和 $\epsilon$ 之间的关系如何演变。幸运的是，由于我们的随机变量是有界的，我们可以通过应用 Hoeffding (1963) 提出的不等式获得有效的有限样本界限：

$$P(\epsilon_\mathcal{D}(f) - \epsilon(f) \geq t) < \exp\left( - 2n t^2 \right).$$

解出最小的数据集大小，允许我们以 95% 的置信度得出结论，估计 $\epsilon_\mathcal{D}(f)$ 与真实误差率 $\epsilon(f)$ 之间的距离 $t$ 不超过 0.01，你会发现大约需要 15,000 个样本，相比上面渐近分析建议的 10,000 个样本。如果你深入研究统计学，你会发现这种趋势通常成立。即使在有限样本中也成立的保证通常稍微保守一些。需要注意的是，在大局中，这些数字并不相差太远，反映出渐近分析即使不是我们可以带到法庭上的保证，通常也能为我们提供有用的粗略估计。

## 测试集重用

在某种意义上，你现在准备好成功地进行实证机器学习研究。几乎所有实用模型都是基于测试集性能开发和验证的，你现在已经成为测试集的大师。对于任何固定的分类器 $f$，你知道如何评估其测试误差 $\epsilon_\mathcal{D}(f)$，并且确切知道可以（和不可以）说些什么关于其总体误差 $\epsilon(f)$。

所以假设你利用这些知识准备训练你的第一个模型 $f_1$。了解你需要对自己的分类器误差率有多大的信心后，你应用上述分析来确定适当的测试集样本数量。此外，假设你吸取了 :numref:`sec_generalization_basics` 的教训，确保通过在验证集上进行所有初步分析、超参数调整甚至在多个竞争模型架构之间进行选择来维护测试集的神圣性。最后，你在测试集上评估你的模型 $f_1$ 并报告一个无偏的总体误差估计及其相关的置信区间。

到目前为止一切似乎进展顺利。然而，那晚你在凌晨 3 点醒来，突然有了一个新的建模方法的灵感。第二天，你编写了新的模型，在验证集上调优其超参数，不仅让新模型 $f_2$ 工作起来，而且其误差率似乎比 $f_1$ 低得多。然而，当你准备最终评估时，兴奋感突然消失了——你没有测试集了！

即使原始测试集 $\mathcal{D}$ 仍然在你的服务器上，你现在面临着两个艰巨的问题。首先，当你收集测试集时，你是在假设仅评估单个分类器 $f$ 的前提下确定所需精度水平的。然而，如果你开始在同一测试集上评估多个分类器 $f_1, ..., f_k$，你必须考虑误发现的问题。以前，你可能有 95% 的把握 $\epsilon_\mathcal{D}(f) \in \epsilon(f) \pm 0.01$ 对于单个分类器 $f$，因此误导性结果的概率仅为 5%。如果有 $k$ 个分类器参与其中，很难保证其中没有任何一个分类器的测试集性能具有误导性。如果有 20 个分类器需要考虑，你可能根本没有能力排除至少有一个分类器收到了误导性评分的可能性。这个问题与多重假设检验有关，尽管统计学中有大量的文献，但它仍然是困扰科学研究的一个持续存在的问题。

如果这还不足以让你担忧，还有另一个特别的理由让你对后续评估的结果产生怀疑。回顾一下，我们对测试集性能的分析基于这样一个假设：分类器是在没有任何接触测试集的情况下选择的，因此我们可以将测试集视为从基础人群中随机抽取的。在这里，你不仅在测试多个函数，随后的函数 $f_2$ 是在观察到 $f_1$ 的测试集性能之后选择的。一旦测试集的信息泄露给了建模者，它就再也不能严格意义上的真正的测试集了。这个问题被称为*自适应过拟合*，最近引起了学习理论家和统计学家的极大兴趣 :cite:`dwork2015preserving`。幸运的是，虽然有可能泄露掉所有保留集的信息，并且理论上的最坏情况很严峻，但这些分析可能过于保守。在实践中，请注意创建真正的测试集，尽可能少地参考它们，在报告置信区间时考虑多重假设检验，并在利益重大且数据集规模较小时更加警惕。在运行一系列基准挑战时，通常好的做法是维护多个测试集，以便每轮之后，旧的测试集可以降级为验证集。

## 统计学习理论

简单地说，*测试集是我们真正拥有的全部*，然而这个事实似乎令人奇怪地感到不满。首先，我们很少拥有*真正的测试集*——除非我们是创建数据集的人，否则其他人很可能已经用自己的分类器评估了我们所谓的“测试集”。即使我们最先使用，很快我们就会感到沮丧，希望能评估我们后续的建模尝试，而不会有无法信任自己数据的不安感觉。此外，即使是真正的测试集也只能告诉我们*事后*一个分类器是否确实推广到了总体，而不能告诉我们是否有理由*先验*期望它应该推广。

带着这些疑虑，你可能现在已经足够准备好看到*统计学习理论*的魅力了，这是机器学习的数学子领域，其研究人员旨在阐明解释基于经验数据训练的模型为何/何时可以/将会推广到未见数据的基本原理。统计学习研究人员的主要目标之一是限定泛化差距，将模型类的性质与数据集中的样本数量联系起来。

学习理论家的目标是限定学习分类器 $f_\mathcal{S}$ 的*经验误差* $\epsilon_\mathcal{S}(f_\mathcal{S})$ 与该相同分类器在基础人群中的真实误差 $\epsilon(f_\mathcal{S})$ 之间的差异，两者都在训练集 $\mathcal{S}$ 上进行训练和评估。这看起来可能类似于我们刚刚解决的评估问题，但有一个主要区别。之前，分类器 $f$ 是固定的，我们只需要一个数据集用于评估目的。事实上，任何固定的分类器都会推广：它在一个（以前未见过的）数据集上的误差是对总体误差的无偏估计。但是，当一个分类器在同一数据集上训练和评估时，我们能说什么呢？我们能否有信心认为训练误差会接近测试误差？

假设我们的学习分类器 $f_\mathcal{S}$ 必须从预先指定的函数集 $\mathcal{F}$ 中选择。回想我们关于测试集的讨论，虽然很容易估计单个分类器的误差，但当我们开始考虑分类器集合时，事情就变得复杂了。即使任何一个（固定的）分类器的经验误差都将以高概率接近其真实误差，一旦我们考虑一个分类器集合，我们就需要担心其中一个分类器可能被严重低估误差的可能性。担心的是我们可能会选择这样一个分类器，从而大大低估了总体误差。此外，即使是线性模型，由于其参数是连续值的，我们通常是从无限类函数中选择（$|\mathcal{F}| = \infty$）。

解决这个问题的一个雄心勃勃的方法是开发分析工具，以证明一致收敛性，即以高概率，类 $f\in\mathcal{F}$ 中的每个分类器的经验误差率将*同时*收敛到其真实误差率。换句话说，我们寻求一种理论原则，使我们能够声明，以至少 $1-\delta$ 的概率（对于某个小的 $\delta$），类 $\mathcal{F}$ 中的所有分类器的误差率 $\epsilon(f)$ 都不会被高估超过某个小量 $\alpha$。显然，我们不能对所有模型类 $\mathcal{F}$ 做出这样的陈述。回想一下总是实现经验误差 0 但在基础人群中从未优于随机猜测的存储机类。

在某种意义上，存储机类太过灵活。不可能有这样的统一收敛结果。另一方面，固定分类器是无用的——它完美地泛化，但既不适合训练数据也不适合测试数据。因此，学习的核心问题历来被框定为更具灵活性（更高方差）的模型类与更具刚性（更高偏差）的模型类之间的权衡，前者更好地拟合训练数据但风险过拟合，后者泛化良好但风险欠拟合。学习理论的一个核心问题是开发适当的数学分析来量化模型在这一谱系中的位置，并提供相应的保证。

在一系列开创性的论文中，Vapnik 和 Chervonenkis 扩展了相对频率收敛的理论，使其适用于更一般的函数类 :cite:`VapChe64,VapChe68,VapChe71,VapChe74b,VapChe81,VapChe91`。这一系列工作的关键贡献之一是 Vapnik-Chervonenkis (VC) 维度，它衡量（一种）模型类的复杂性（灵活性）。此外，他们的一个关键结果限定了经验误差与总体误差之间的差异，该差异取决于 VC 维度和样本数量：

$$P\left(R[p, f] - R_\textrm{emp}[\mathbf{X}, \mathbf{Y}, f] < \alpha\right) \geq 1-\delta
\ \textrm{ for }\ \alpha \geq c \sqrt{(\textrm{VC} - \log \delta)/n}.$$

这里 $\delta > 0$ 是违反边界条件的概率，$\alpha$ 是泛化差距的上限，$n$ 是数据集大小。最后，$c > 0$ 是仅取决于可发生损失尺度的常数。该边界的用途之一可能是插入所需的 $\delta$ 和 $\alpha$ 值以确定需要收集多少样本。VC 维度量化了可以为最大数量的数据点分配任意（二进制）标记，并为每个找到模型类中的某个模型 $f$ 与该标记相匹配的最大数据点数。例如，$d$ 维输入上的线性模型具有 $d+1$ 的 VC 维度。很容易看出，一条直线可以为二维中的三个点分配任何可能的标记，但不能为四个点分配。不幸的是，对于更复杂的模型，理论往往过于悲观，获得这种保证通常需要远远多于实际需要的样本来达到所需的误差率。还要注意，固定模型类和 $\delta$ 后，我们的误差率再次以通常的 $\mathcal{O}(1/\sqrt{n})$ 速率衰减。似乎不太可能在 $n$ 方面做得更好。然而，当我们改变模型类时，VC 维度可能会呈现泛化差距的悲观图景。

## 总结

评估模型最直接的方法是咨询由以前未见过的数据组成的测试集。测试集评估提供了真实误差的无偏估计，并且随着测试集的增长，以所需的 $\mathcal{O}(1/\sqrt{n})$ 速率收敛。我们可以基于精确的渐近分布提供近似的置信区间，或者基于（更为保守的）有限样本保证提供有效的有限样本置信区间。事实上，测试集评估是现代机器学习研究的基石。然而，测试集很少是真正的测试集（多次被多个研究者使用）。一旦相同的测试集被用来评估多个模型，控制误发现就变得困难。这在理论上可能造成巨大的问题。在实践中，这个问题的重要性取决于所涉及的保留集的大小以及它们仅仅是用来选择超参数还是更直接地泄露信息。尽管如此，谨慎的做法是策划真正的测试集（或多个），并在使用它们时尽可能保守。

希望提供一个更令人满意的解决方案，统计学习理论家已经开发了保证模型类均匀收敛的方法。如果确实每个模型的经验误差同时收敛到其真实误差，那么我们可以自由选择表现最佳的模型，最小化训练误差，知道它在保留数据上的表现也会相似。至关重要的是，任何此类结果都必须依赖于模型类的某些属性。Vladimir Vapnik 和 Alexey Chernovenkis 引入了 VC 维度，提出了对所有 VC 类模型适用的一致收敛结果。该类中所有模型的经验误差（同时）保证接近其真实误差，并且保证以 $\mathcal{O}(1/\sqrt{n})$ 速率变得更接近。继革命性的 VC 维度发现之后，提出了多种替代的复杂度度量，每种度量都促进了类似的泛化保证。请参阅 :citet:`boucheron2005theory` 以详细了解几种高级的功能复杂度度量方法。不幸的是，虽然这些复杂度度量已成为统计理论中广泛使用的工具，但它们在直接应用于解释深度神经网络为何泛化方面却显得无力。深度神经网络通常具有数百万（或更多）参数，并且可以轻松地为大量点分配随机标签。尽管如此，它们在实际问题上泛化得很好，并且令人惊讶的是，当它们更大更深时，尽管具有更高的 VC 维度，它们往往泛化得更好。在下一章中，我们将重新讨论深度学习背景下的泛化。

## 练习

1. 如果我们希望将固定模型 $f$ 的误差估计到 $0.0001$ 以内，并且概率大于 99.9%，我们需要多少样本？
1. 假设其他人拥有一个带标签的测试集 $\mathcal{D}$，但只提供未标记的输入（特征）。现在假设你只能通过运行模型 $f$（对模型类没有限制）来访问测试集标签，每次在未标记的输入上运行模型并接收相应的误差 $\epsilon_\mathcal{D}(f)$。在泄漏整个测试集之前，你需要评估多少模型，从而无论你的真实误差如何，都能表现出 0 的误差？
1. 五次多项式的 VC 维度是多少？
1. 二维数据上轴对齐矩形的 VC 维度是多少？

[讨论](https://discuss.d2l.ai/t/6829)